# Data Aggregation DRIP

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## read in data 
RAPM = pd.read_csv("RAPM_Predictions_All.csv").drop('Unnamed: 0', axis=1)

In [ ]:
## grab only columns needed 
r = RAPM[['playerid','player','teamname','minutesplayed','season','RAPM_Off_Prediction','RAPM_Def_Prediction']]

## Condense the RAPM Predcitions so there is only one per season 
did average and added percentage 

### Added Percentage of all of them

In [ ]:
## these players are in autostats data
players = [1176246, 1343009, 1350414, 1350608, 1350808, 1350935, 1352114, 1374251]
z = r[r['playerid'].isin(players)]
z['season'] = z['season'].replace(0,2022)

r = r.append(z)

In [ ]:
## there is a bunch of seasons that are zero so going to drop it for now 
r = r[r['season'] > 0].reset_index(drop=True)

In [ ]:
## sums up all AutoStats Games a player has 
a = r.merge(pd.DataFrame(r.groupby(['playerid', 'season']) ['minutesplayed'].sum()), on=['playerid', 'season'])

## creates a percentage of each player for each season
r['percent'] = (a['minutesplayed_x']/a['minutesplayed_y'])

In [ ]:
## creates the columns we need to multiple the percent by
num_cols = r.drop(['minutesplayed', 'playerid','player','teamname','season', 'percent'], axis=1)

In [ ]:
## gets the percent of each variable based on game
for i in num_cols:
    r[i] = r[i] * r['percent']

In [ ]:
## adds up the lines per each player per season
r = r.groupby(['playerid', 'season']).agg('sum').reset_index()

In [ ]:
## creates a variable that gives what number year it is for each season
r['rank'] = r.groupby(['playerid'])['season'].rank('dense', ascending=True)

In [ ]:
## drop minuteplayed 
r = r.drop(['minutesplayed'], axis=1)

### Functions to Switch the rows to columns 

In [ ]:
""" 
Function that changes the rows of each year to columns

INPUT: the year (rank), the data needed, anything that needs to be dropped
OUTPUT: the dataframe for that year 

"""
def rows_to_columns(year, data, drop):
    rapm_year = data[data['rank'] ==year].rename({'season':'year' + str(year),'RAPM_Off_Prediction': 'DRIP_Off_YEAR' + str(year),
                                       'RAPM_Def_Prediction': 'DRIP_Def_YEAR'+ str(year)}, axis=1).drop(drop, axis=1)
    return rapm_year

In [ ]:
""" 
Function that merges two dataframes as an outer merge

INPUT: two dataframes
OUTPUT: merged dataframe 

"""
def merge(df1, df2, df3):
    merged = pd.merge(pd.merge(df1, df2, on = 'playerid', how='outer'), df3, on='playerid', how='outer')
    return merged

In [ ]:
## all the years (11)
r1 = rows_to_columns(1, r, ['rank', 'percent'])
r2 = rows_to_columns(2, r, ['rank', 'percent'])
r3 = rows_to_columns(3, r, ['rank', 'percent'])
r4 = rows_to_columns(4, r, ['rank', 'percent'])
r5 = rows_to_columns(5, r, ['rank', 'percent'])
r6 = rows_to_columns(6, r, ['rank', 'percent'])
r7 = rows_to_columns(7, r, ['rank', 'percent'])
r8 = rows_to_columns(8, r, ['rank', 'percent'])
r9 = rows_to_columns(9, r, ['rank', 'percent'])
r10 = rows_to_columns(10, r, ['rank', 'percent'])
r11 = rows_to_columns(11, r, ['rank', 'percent'])

In [ ]:
## merges 
r123 = merge(r1, r2, r3)
r456 = merge(r4, r5, r6)
r789 = merge(r7, r8, r9)
r19 = merge(r123, r456, r789)
rapm_merged1 = merge(r19, r10, r11)

In [ ]:
rapm_merged1.to_csv("RAPM_Added_year.csv")

### Average

In [ ]:
rapm = RAPM[['playerid','player','season','RAPM_Off_Prediction','RAPM_Def_Prediction']]

In [ ]:
## these players are in autostats data
players = [1176246, 1343009, 1350414, 1350608, 1350808, 1350935, 1352114, 1374251]
z = rapm[rapm['playerid'].isin(players)]
z['season'] = z['season'].replace(0,2022)

rapm = rapm.append(z)

In [ ]:
## there is a bunch of seasons that are zero so going to drop it for now 
rapm = rapm[rapm['season'] > 0].reset_index(drop=True)

In [ ]:
## takes the average RAPM per season
rapm = rapm.groupby(['playerid', 'season']).agg({'RAPM_Off_Prediction':'mean','RAPM_Def_Prediction':'mean'}).reset_index()

In [ ]:
## creates a variable that gives what number year it is for each season
rapm['rank'] = rapm.groupby(['playerid'])['season'].rank('dense', ascending=True)

In [ ]:
## all the years (11)
rapm1 = rows_to_columns(1, rapm, ['rank'])
rapm2 = rows_to_columns(2, rapm, ['rank'])
rapm3 = rows_to_columns(3, rapm, ['rank'])
rapm4 = rows_to_columns(4, rapm, ['rank'])
rapm5 = rows_to_columns(5, rapm, ['rank'])
rapm6 = rows_to_columns(6, rapm, ['rank'])
rapm7 = rows_to_columns(7, rapm, ['rank'])
rapm8 = rows_to_columns(8, rapm, ['rank'])
rapm9 = rows_to_columns(9, rapm, ['rank'])
rapm10 = rows_to_columns(10, rapm, ['rank'])
rapm11 = rows_to_columns(11, rapm, ['rank'])

In [ ]:
## merges 
rapm123 = merge(rapm1, rapm2, rapm3)
rapm456 = merge(rapm4, rapm5, rapm6)
rapm789 = merge(rapm7, rapm8, rapm9)
rapm19 = merge(rapm123, rapm456, rapm789)
rapm_merged2 = merge(rapm19, rapm10, rapm11)

In [ ]:
rapm_merged2.to_csv('RAPM_AVG_year.csv')